<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_06/blob/equation_function/Revised_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

# Get domain parameters
def getInputs():
  '''
    Gets all inputs needed for the equation

    Returns:
        Float : Domain length
        Float : simulation time
        Float : river velocity
        Integer : Grid points Nx
        Integer : Grid points Nt
        Float : step size dx
        Float : step size dt
        Float : decay constant
    '''

  L = float(input("Enter domain length (m): "))
  T = float(input("Enter total simulation time (s): "))
  U = float(input("Enter river velocity U (m/s): "))
  k = float(input("Enter the decay constant (if no decay enter 0) "))
  # Choose input method
  print("\nGrid setup options:")
  print("1. Input number of grid points (Nx, Nt)")
  print("2. Input step sizes (Δx, Δt)")
  choice = input("Enter 1 or 2: ")

  if choice == "1":
      Nx = int(input("Enter Nx (number of spatial points): "))
      Nt = int(input("Enter Nt (number of time steps): "))
      dx = L / (Nx - 1)
      dt = T / (Nt - 1)
  else:
      dx = float(input("Enter Δx (spatial step size in m): "))
      dt = float(input("Enter Δt (time step size in s): "))
      Nx = int(L / dx) + 1
      Nt = int(T / dt) + 1

  return L,T,U,Nx,Nt,dx,dt,k



def concovertime(L,T,U,Nx,Nt,dx,dt,decay=0,random=0):
  '''
  Gets all inputs needed for the equation

    Parameters:
        L(float)
        T(float)
        U(float)
        Nx(Int)
        Nt(Int)
        dx(float)
        dt(float)
        decay(float)
        random(float)
  '''
  # Create grids
  x = np.linspace(0, L, Nx)  # Spatial grid: [0, Δx, 2Δx, ..., L]
  t = np.linspace(0, T, Nt)  # Time grid: [0, Δt, 2Δt, ..., T]
  theta_new = np.zeros(Nx-1)  # array for thetas
  theta_old = np.zeros(Nx-1)

  #arrays for matrix coefficients
  A = np.zeros(Nx-1)
  B = np.zeros(Nx-1)
  F = np.zeros(Nx-1)

  history = np.zeros((Nt, Nx-1))  # this was changed
  history[0, 0] = 250.0

  #varying velocity 10% random perturbation
  np.random.seed(1)
  u = U * (1 + 0.1 * np.random.randn(Nx-1)) #-1 was added

  # Main simulation loop for test case 1
  for j in range(1, Nt):
    theta_new[:] = 0.0
    theta_new[0] = 250.0 * np.exp(-decay * j * Nt)


    random_variable = np.random.random((int(L / dx+1)))
    random_speed = (1 - random/100 + random/50 * random_variable) * U

    #store values in array A and B with varying u
    for i in range(Nx-1):
        A[i] = (1.0 / dt) + (u[i] / dx)
        B[i] = - u[i] / dx

    #RHS vector F
    for i in range(Nx-2):
        F[i] = (1/dt)*theta_old[i+1]

    #forward subsitution
    for I in range(1,Nx-1):
        theta_new[I] = (1/A[I-1])*(F[I-1]-B[I-1]*theta_new[I-1])

    #save solution

    history[j] = theta_new.copy()  # this was left as a comment
    theta_old[:] = theta_new[:]


  print("A[0:3] =", A[:3])
  print("B[0:3] =", B[:3])

  print("theta_new[0:5] =", theta_new[:10])

  #graphs
  matplotlib.rcParams["animation.html"] = "jshtml"
  matplotlib.rcParams['figure.dpi'] = 150

  plt.ioff()  # turn interactive mode off
  fig, ax = plt.subplots(figsize=(8,5))

  def animate(frame):
      ax.cla()  # clear previous frame
      ax.plot(x[:-1], history[frame, :], color='blue')
      ax.set_xlabel("Distance downstream (m)")
      ax.set_ylabel("Concentration (µg/m³)")
      ax.set_ylim(0, 260)
      ax.grid(True)
      ax.set_title(f"Pollutant concentration at t = {frame*dt:.1f} s")

  # Create the animation
  ani = FuncAnimation(fig, animate, frames=Nt, interval=50)

  # Display animation in Jupyter
  return ani


L,T,U,Nx,Nt,dx,dt,k = getInputs()
concovertime(L,T,U,Nx,Nt,dx,dt,k)


Enter domain length (m): 20
Enter total simulation time (s): 300
Enter river velocity U (m/s): 0.1
Enter the decay constant (if no decay enter 0) 0

Grid setup options:
1. Input number of grid points (Nx, Nt)
2. Input step sizes (Δx, Δt)
Enter 1 or 2: 2
Enter Δx (spatial step size in m): 0.2
Enter Δt (time step size in s): 10
A[0:3] = [0.68121727 0.56941218 0.57359141]
B[0:3] = [-0.58121727 -0.46941218 -0.47359141]
theta_new[0:5] = [250. 250. 250. 250. 250. 250. 250. 250. 250. 250.]
